# HR Attendance Processing Notebook

This notebook converts the Streamlit app into step-by-step Jupyter code so you can test each part before finalizing the Streamlit version.

In [1]:
import warnings
import io
import pandas as pd
import numpy as np
#import gspread
#from google.oauth2.service_account import Credentials
#from utility import connect_gsheet, get_dataframe, gs_client
from utility_attendance import (
    stepwise_file_upload, split_file, merge_files, move_columns,
    weighted_sum_and_replace_columns, calculate_leave_summary_with_wd_leaves,
    calculate_working_days
)
from datetime import datetime, timedelta, date
warnings.filterwarnings("ignore")

In [2]:
#from datetime import datetime, date

In [65]:
%run utility_attendance.py

In [4]:
no_working_days = 20

## Step 1: Read files and merge

In [5]:
try:
    client = connect_gsheet()
    emp_df = get_dataframe(client, "<your_sheet_id>", "users")
    print("✅ Google Sheets connection successful")
    display(emp_df.head())
except Exception as e:
    print(f"❌ Error connecting to Google Sheets: {e}")

❌ Error connecting to Google Sheets: name 'connect_gsheet' is not defined


In [60]:
# 
df_gimt = pd.read_excel('./data/GIMT_MonthlyAttendanceSummaryReport (28).xlsx')
df_admin = pd.read_excel('./data/Admin_MonthlyAttendanceSummaryReport(45).xlsx')
df_gips = pd.read_excel('./data/GIPS_MonthlyAttendanceSummaryReport(45).xlsx')
df_leave_erp = pd.read_csv('./data/Leave report sep25.csv',skiprows=6, encoding='windows-1252')
exempted_file = pd.ExcelFile('./data/template_exempted_sep.xlsx')

### Reading EMP data from ERP
df_emp = pd.read_csv('./data/emp_master_data.csv', skiprows=6, encoding='windows-1252')

In [61]:
df_gimt_all, df_gimt_in, df_gimt_out = split_file(df_gimt)
df_gimt_merged = merge_files(df_gimt_in, df_gimt_out, no_working_days)

df_gips_all, df_gips_in, df_gips_out = split_file(df_gips)
df_gips_merged = merge_files(df_gips_in, df_gips_out, no_working_days)



In [66]:
df_admin_all, df_admin_in, df_admin_out = split_file(df_admin)
df_admin_merged = merge_files_staffs(df_admin_in, df_admin_out, no_working_days)

TypeError: 'int' object is not iterable

In [8]:
df_gips_merged.head(3)

,Emp Id,Names,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_late,Absent,Working Days,Present
0,GCU020004,Dr. Dipankar,[],[],[],[],[],[],0,0.0,20,20.0
1,GCU020007,Dr Smriti Rekha Chanda,[],[],[],[],[],[8_28],0,1.0,20,19.0
2,GCU020018,Dr. Bhanu P.,"[8_26, 9_08, 9_15, 9_16]",[],"[8_26, 8_27, 9_01, 9_02, 9_03, 9_04, 9_05, 9_0...",[9_10],"[9_02, 9_13, 9_01, 9_15, 8_27, 9_04, 9_08, 9_1...","[9_12, 9_11, 8_30, 8_28, 8_29]",4,12.0,20,8.0


In [9]:
df_admin_merged.sample(7)

,Emp Id,Names,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_late,Absent,Working Days,Present
125,GCU030163,Sagar,[],[],"[8_27, 9_03, 9_17]","[8_27, 9_03]",[9_17],"[8_31, 8_30, 9_14, 9_06, 9_07, 9_20]",0,7.5,20,12.5
8,GCU010034,Manish,"[8_27, 9_01]",[],"[9_11, 9_18]","[9_11, 9_18]",[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",2,7.0,20,13.0
38,GCU030025,Latif,[],[],"[8_26, 8_27, 8_28, 8_29, 8_30, 9_01, 9_03, 9_0...",[],"[9_12, 9_13, 9_01, 9_10, 9_15, 8_27, 9_04, 9_1...","[8_31, 9_14, 9_06, 9_07, 9_16, 9_17, 9_20]",0,15.0,20,5.0
75,GCU030083,Sufia,[9_03],[],[9_17],[],[9_17],"[8_31, 9_14, 9_07, 9_06, 9_20, 9_05]",1,6.5,20,13.5
20,GCU030004,Nur Islam,[9_17],[],[],[],[],"[8_31, 9_14, 9_06, 9_07, 9_20]",1,5.0,20,15.0
49,GCU030039,Merina,"[8_31, 9_06, 9_14, 9_20]","[8_31, 9_06, 9_14, 9_20]","[9_14, 9_17]",[],"[9_14, 9_17]","[9_12, 9_08, 9_19, 9_07, 8_29, 9_09]",4,7.0,20,13.0
89,GCU030102,Jit,[],[],[],[],[],"[9_11, 9_10, 9_08, 9_04, 9_19, 9_14, 8_28, 9_1...",0,26.0,20,-6.0


In [10]:
df_admin_merged.shape

(143, 12)

### Step 1.1: Faculty Detailed view (GIMT + GIPS only)

In [11]:
#df_fac_all = pd.concat([splits["GIMT"]["all"], splits["GIPS"]["all"]], ignore_index=True)
df_fac_detail = pd.concat([df_gimt_all, df_gips_all], ignore_index=True)
df_fac_conso = pd.concat([df_gimt_merged, df_gips_merged], ignore_index=True)
df_fac_detail.fillna(0, inplace=True)
df_fac_conso.fillna(0, inplace=True)


In [12]:
df_fac_conso.head()

,Emp Id,Names,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_late,Absent,Working Days,Present
0,GCU010013,Ms. Raisa,[8_28],"[8_27, 9_13]","[8_27, 8_28, 9_13]",[8_28],"[9_13, 8_27]","[9_12, 9_18]",1,3.5,20,16.5
1,GCU010017,Ms. Ananya,[8_27],[],[],[],[],[9_03],1,1.0,20,19.0
2,GCU020001,Dr. Lipi,[],[],[],[],[],"[9_11, 8_26]",0,2.0,20,18.0
3,GCU020002,Dr. Swarnali,[],[],[],[],[],"[8_28, 8_29]",0,2.0,20,18.0
4,GCU020003,Dr. Kajal,[],[],[],[],[],[],0,0.0,20,20.0


In [13]:
df_admin_detail = df_admin_all.copy()
df_admin_detail.fillna(0, inplace=True)  

df_admin_conso = df_admin_merged.copy()
df_admin_conso.fillna(0, inplace=True)  

In [14]:
df_admin_conso.head()

,Emp Id,Names,late_flags,early_flags,half_day_flags,AM_abs,PM_abs,days_abs,No_of_late,Absent,Working Days,Present
0,GCU010004,Kishore Kr.,"[8_28, 8_29, 8_30, 9_02, 9_03, 9_05, 9_08, 9_0...",[],"[8_26, 8_27, 8_28, 8_29, 9_01, 9_02, 9_04, 9_0...","[9_12, 9_01, 9_11, 9_10, 9_15, 8_27, 9_04, 8_2...",[9_08],"[8_31, 9_14, 9_06, 9_07, 9_16, 9_17, 9_20]",14,13.5,20,6.5
1,GCU010005,Dwipen,"[8_30, 9_02, 9_04, 9_09, 9_12, 9_15, 9_17, 9_19]",[9_17],"[9_17, 9_17]",[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",8,6.0,20,14.0
2,GCU010011,Nripen Ch,[],[],[],[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",0,6.0,20,14.0
3,GCU010012,Sourabh Mazinder,[],[],[],[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",0,6.0,20,14.0
4,GCU010024,Mridul,[9_09],[],[9_15],[],[9_15],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",1,6.5,20,13.5


In [15]:
col_to_rename = {'AM_abs':'actual_AM_abs','PM_abs':'actual_PM_abs','days_abs':'actual_days_abs','No_of_late':'actual_No_of_late'}

df_fac_conso.rename(columns=col_to_rename, inplace=True)
df_admin_conso.rename(columns=col_to_rename, inplace=True)


In [16]:
#df_fac_conso.columns
# current dataframe: df
desired_order = [
    'Emp Id', 'Names',
    'Working Days', 'Present', 'Absent',
    'actual_AM_abs', 'actual_PM_abs', 'actual_days_abs',
    'half_day_flags', 'late_flags', 'early_flags',
    'actual_No_of_late'
]

# Keep only those that exist in df (to avoid KeyError)
desired_order = [col for col in desired_order if col in df_fac_conso.columns]

df_fac_conso = df_fac_conso[desired_order]
df_admin_conso = df_admin_conso[desired_order]

In [17]:
df_admin_conso.head()

,Emp Id,Names,Working Days,Present,Absent,actual_AM_abs,actual_PM_abs,actual_days_abs,half_day_flags,late_flags,early_flags,actual_No_of_late
0,GCU010004,Kishore Kr.,20,6.5,13.5,"[9_12, 9_01, 9_11, 9_10, 9_15, 8_27, 9_04, 8_2...",[9_08],"[8_31, 9_14, 9_06, 9_07, 9_16, 9_17, 9_20]","[8_26, 8_27, 8_28, 8_29, 9_01, 9_02, 9_04, 9_0...","[8_28, 8_29, 8_30, 9_02, 9_03, 9_05, 9_08, 9_0...",[],14
1,GCU010005,Dwipen,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]","[9_17, 9_17]","[8_30, 9_02, 9_04, 9_09, 9_12, 9_15, 9_17, 9_19]",[9_17],8
2,GCU010011,Nripen Ch,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[],[],[],0
3,GCU010012,Sourabh Mazinder,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[],[],[],0
4,GCU010024,Mridul,20,13.5,6.5,[],[9_15],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[9_15],[9_09],[],1


## Step 2: Merge with ERP employee data

In [18]:
df_emp.head(3)

,Serial No,Employee ID,Name,Department,Group Name,User Type,Designation,Joining Date,Email ID,Mobile No.,Date of Birth,Sub Department,Academy
0,1,APPLICANT_PORTAL_USER,APP_FIRST_NAME APP_MIDDLE_NAME APP_LAST_NAME,-,Admin,-,-,-,-,-,-,-,Girijananda Chowdhury University
1,2,mis,MIS Admin,Administration,Admin,-,-,-,mayank.pandya@serosoft.in,+91-1111111111,18/03/1999,-,Girijananda Chowdhury University
2,3,EMP4,Lokesh Yadav,School of Allied Health Sciences,Admin,-,-,-,lokesh.yadav@serosoft.in,+91-1111111111,09/01/1989,-,Girijananda Chowdhury University


In [19]:
emp_df = df_emp[['Employee ID','Name','Designation','Department']]
emp_df = emp_df.rename(columns={'Employee ID':'Emp Id'})

In [20]:
#df_fac_detail.rename(columns={'Emp Id':'Emp ID'}, inplace=True)
#df_admin_detail.rename(columns={'Emp Id':'Emp ID'},inplace=True)

In [21]:
df_fac_detail_ID = pd.merge(df_fac_detail, emp_df, how='left', on='Emp Id')
df_fac_detail_ID = move_columns(df_fac_detail_ID, {'Name':1,'Designation':2,'Department':3})
df_fac_detail_ID = df_fac_detail_ID.drop('Names', axis=1)

df_admin_detail_ID = pd.merge(df_admin_detail, emp_df, how='left', on='Emp Id')
df_admin_detail_ID = move_columns(df_admin_detail_ID, {'Name':1,'Designation':2,'Department':3})
df_admin_detail_ID = df_admin_detail_ID.drop('Names', axis=1)


In [22]:
df_fac_conso.head()

,Emp Id,Names,Working Days,Present,Absent,actual_AM_abs,actual_PM_abs,actual_days_abs,half_day_flags,late_flags,early_flags,actual_No_of_late
0,GCU010013,Ms. Raisa,20,16.5,3.5,[8_28],"[9_13, 8_27]","[9_12, 9_18]","[8_27, 8_28, 9_13]",[8_28],"[8_27, 9_13]",1
1,GCU010017,Ms. Ananya,20,19.0,1.0,[],[],[9_03],[],[8_27],[],1
2,GCU020001,Dr. Lipi,20,18.0,2.0,[],[],"[9_11, 8_26]",[],[],[],0
3,GCU020002,Dr. Swarnali,20,18.0,2.0,[],[],"[8_28, 8_29]",[],[],[],0
4,GCU020003,Dr. Kajal,20,20.0,0.0,[],[],[],[],[],[],0


In [23]:
# there was error till here

In [24]:
df_fac_conso_ID = pd.merge(df_fac_conso, emp_df, how='left', on='Emp Id')
df_fac_conso_ID = move_columns(df_fac_conso_ID, {'Name':1,'Designation':2,'Department':3})
df_fac_conso_ID = df_fac_conso_ID.drop('Names', axis=1)

df_admin_conso_ID = pd.merge(df_admin_conso, emp_df, how='left', on='Emp Id')
df_admin_conso_ID = move_columns(df_admin_conso_ID, {'Name':1,'Designation':2,'Department':3})
df_admin_conso_ID = df_admin_conso_ID.drop('Names', axis=1)


In [25]:
df_admin_conso_ID.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_AM_abs,actual_PM_abs,actual_days_abs,half_day_flags,late_flags,early_flags,actual_No_of_late
0,GCU010004,Kishore Kumar Choudhury,-,Administration,20,6.5,13.5,"[9_12, 9_01, 9_11, 9_10, 9_15, 8_27, 9_04, 8_2...",[9_08],"[8_31, 9_14, 9_06, 9_07, 9_16, 9_17, 9_20]","[8_26, 8_27, 8_28, 8_29, 9_01, 9_02, 9_04, 9_0...","[8_28, 8_29, 8_30, 9_02, 9_03, 9_05, 9_08, 9_0...",[],14
1,GCU010005,Dwipen Das,Administration,Administration,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]","[9_17, 9_17]","[8_30, 9_02, 9_04, 9_09, 9_12, 9_15, 9_17, 9_19]",[9_17],8
2,GCU010011,Nripen Chandra Barman,-,Administration,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[],[],[],0
3,GCU010012,Sourobh Mazinder Barua,-,Administration,20,14.0,6.0,[],[],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[],[],[],0
4,GCU010024,Mridul Das,-,Administration,20,13.5,6.5,[],[9_15],"[8_31, 9_14, 9_06, 9_07, 9_17, 9_20]",[9_15],[9_09],[],1


### SAVE 1 & 2 (Detailed and Consolidated data)

In [26]:
# 1
print(f"This is the detailed faculty biometric data {df_fac_detail_ID.shape}")
print(f"This is the detailed admin biometric data {df_admin_detail_ID.shape}")

# 2
print(f"This is the consolidate faculty biometric data {df_fac_conso_ID.shape}")
print(f"This is the consolidate admin biometric data {df_admin_conso_ID.shape}")

This is the detailed faculty biometric data (161, 56)
This is the detailed admin biometric data (143, 56)
This is the consolidate faculty biometric data (161, 14)
This is the consolidate admin biometric data (143, 14)


### 2.1 Handling half days

In [27]:
#df_admin_with_ID.to_excel("./output/staff_bio_data.xlsx", index=False)

In [28]:
# helper for safety
#def safe_len(x):
#    return len(x) if isinstance(x, (list, str)) else 0

In [29]:
df_fac_actual = df_fac_conso_ID.copy()
df_admin_actual = df_admin_conso_ID.copy()

# handling half days
df_fac_actual['actual_half_day'] = df_fac_actual.apply(lambda x: len(x['actual_AM_abs'])+len(x['actual_PM_abs']),axis=1)
df_admin_actual['actual_half_day'] = df_admin_actual.apply(lambda x: len(x['actual_AM_abs'])+len(x['actual_PM_abs']),axis=1)

# handling full days
df_fac_actual['actual_full_day'] = df_fac_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)
df_admin_actual['actual_full_day'] = df_admin_actual.apply(lambda x: len(x['actual_days_abs']),axis=1)

In [31]:
col_to_select = ['Emp Id', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent',\
                               'actual_half_day', 'actual_full_day', 'actual_No_of_late']

df_fac_actual = df_fac_actual[col_to_select]
df_admin_actual = df_admin_actual[col_to_select]


In [ ]:
df_fac_actual.head(3)

In [ ]:
df_admin_actual.head()

## Step 3: Exempted leave adjustments

In [32]:
#df_exempted = pd.read_excel('./data/template_exempted.xlsx')
df_exempted = process_exempted_leaves(exempted_file)

In [33]:
df_exempted.head()

,Emp Id,Name,late_count,half_day_count,full_day_count
0,GCU020001,Dr. Lipi Goswami,0.0,0.0,0.0
1,GCU020002,Dr. Swarnali Pathak,0.0,0.0,0.0
2,GCU020003,Dr. Kajal Dutta,0.0,0.0,0.0
3,GCU020004,Dr. Dipankar Saha,0.0,0.0,0.0
4,GCU020006,Dr. Anindita Bora,0.0,0.0,0.0


In [ ]:
#df_exempted.to_excel('/output/staff_exempted.xlsx')
#df_exempted.to_excel("./output/staff_exempted.xlsx", index=False)

In [34]:
df_exempted.rename(columns={'late_count':'exempt_late','half_day_count':'exempt_HD','full_day_count':'exempt_FD'}, inplace=True)
df_exempted.drop('Name',axis=1,inplace=True)
#df_fac_bio_leave = pd.merge(df_fac_with_ID, df_exempted, how='left', on=['Emp ID']).fillna(0)

In [ ]:
df_exempted.head(3)

In [35]:
df_exempted[df_exempted['Emp Id']=='GCU010017']

,Emp Id,exempt_late,exempt_HD,exempt_FD
123,GCU010017,2.0,0.0,0.0


### 3.1 Merging Actual and Exemped Leaves

In [36]:
df_fac_actual_exempted = pd.merge(df_fac_actual,df_exempted , how='left', on=['Emp Id'])
df_fac_actual_exempted.fillna(0, inplace=True)

df_admin_actual_exempted = pd.merge(df_admin_actual,df_exempted , how='left', on=['Emp Id'])
df_admin_actual_exempted.fillna(0, inplace=True)


In [ ]:
df_fac_actual_exempted.head(5)

In [ ]:
df_admin_actual_exempted.head()

### 3.2 Calculating the balance Actual and Exempted Leaves

In [ ]:
#df_fac_attend.head(3)

In [37]:
df_fac_actual_exempted['Half Days'] = np.maximum(df_fac_actual_exempted['actual_half_day'] - df_fac_actual_exempted['exempt_HD'],0)
df_fac_actual_exempted['Full Days'] = np.maximum(df_fac_actual_exempted['actual_full_day'] - df_fac_actual_exempted['exempt_FD'],0)
df_fac_actual_exempted['Late'] = np.maximum(df_fac_actual_exempted['actual_No_of_late'] - df_fac_actual_exempted['exempt_late'],0)

df_admin_actual_exempted['Half Days'] = np.maximum(df_admin_actual_exempted['actual_half_day'] - df_admin_actual_exempted['exempt_HD'],0)
df_admin_actual_exempted['Full Days'] = np.maximum(df_admin_actual_exempted['actual_full_day'] - df_admin_actual_exempted['exempt_FD'],0)
df_admin_actual_exempted['Late'] = np.maximum(df_admin_actual_exempted['actual_No_of_late'] - df_admin_actual_exempted['exempt_late'],0)

In [ ]:
df_fac_actual_exempted.head()

In [ ]:
df_admin_actual_exempted.head()

### SAVE 3 (Actual and Exempted Adjusted)

In [38]:
# 3
print(f"This is the adjusted faculty actual and exempted data {df_fac_actual_exempted.shape}")
print(f"This is the adjusted admin actual and exempted data {df_admin_actual_exempted.shape}")

This is the adjusted faculty actual and exempted data (161, 16)
This is the adjusted admin actual and exempted data (143, 16)


In [39]:
col_to_select = ['Emp Id', 'Name', 'Designation', 'Department', 'Working Days', 'Present', 'Absent','Half Days','Full Days','Late']
df_fac_attend_adjusted = df_fac_actual_exempted[col_to_select]
df_admin_attend_adjusted = df_admin_actual_exempted[col_to_select]

In [40]:
df_fac_attend_adjusted.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Half Days,Full Days,Late
0,GCU010013,Raisa Choudhury,-,Administration,20,16.5,3.5,3.0,2.0,1.0
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0.0,1.0,0.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,0.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0.0,0.0,0.0


In [ ]:
df_admin_attend_adjusted.head()

## Step 4: ERP Leave integration

In [ ]:
df_leave_erp.head(3)

In [ ]:
df_leave_erp.info()

In [41]:
df_leave_erp["From Date"] = df_leave_erp["From Date"].apply(preprocess_date)
df_leave_erp["To Date"] = df_leave_erp["To Date"].apply(preprocess_date)

df_leave_erp["From Date"] = pd.to_datetime(df_leave_erp["From Date"], errors='coerce')
df_leave_erp["To Date"] = pd.to_datetime(df_leave_erp["To Date"], errors='coerce')

In [42]:
# corrected leaves
df_leave_erp_summary = calculate_leave_summary_with_wd_leaves(df_leave_erp, calculate_working_days(df_gimt_in)) 
df_leave_erp_summary.fillna(0, inplace=True)


In [43]:
df_leave_erp_summary[df_leave_erp_summary['Emp Id']=='GCU030123']

,Emp Id,Name,Maternity Leave,Earned Leave,Casual Leave,Duty Leave,Sick Leave,Vacation Leave,Total WD leaves
107,GCU030123,Sumit Basfor,0.0,6.0,0.0,0.0,0.0,0.0,5.0


### 4.1 SAVE 4 (leave approve from ERP)

In [44]:
print(f"This is the Leave approved through ERP {df_leave_erp_summary.shape}")

This is the Leave approved through ERP (191, 9)


In [ ]:
df_leave_erp_summary.head(3)

In [45]:
df_leave_erp_summary['Approved leaves (ERP)'] = df_leave_erp_summary['Total WD leaves'] + df_leave_erp_summary['Casual Leave']
cols_to_drop = [
    "Casual Leave", "Sick Leave", "Duty Leave", "Vacation Leave",
    "Maternity Leave", "Earned Leave", "Paternity Leave",'Total WD leaves'
]

df_leave_compact = df_leave_erp_summary.drop(columns=[c for c in cols_to_drop if c in df_leave_erp_summary.columns], errors="ignore")

In [46]:
#df_leave_compact = df_leave_compact[['Emp ID','Name','Total WD leaves','Extraordinary Leave']]
df_leave_compact = df_leave_compact[['Emp Id','Name','Approved leaves (ERP)']]

In [ ]:
df_leave_compact.head()

In [47]:
df_leave_compact.drop(columns='Name', axis=1,inplace=True)

df_fac_report = pd.merge(df_fac_actual_exempted, df_leave_compact , how='left', on=['Emp Id'])
df_fac_report.fillna(0, inplace=True)

df_admin_report = pd.merge(df_admin_actual_exempted, df_leave_compact , how='left', on=['Emp Id'])
df_admin_report.fillna(0, inplace=True)


In [48]:
df_fac_report.head()

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,actual_half_day,actual_full_day,actual_No_of_late,exempt_late,exempt_HD,exempt_FD,Half Days,Full Days,Late,Approved leaves (ERP)
0,GCU010013,Raisa Choudhury,-,Administration,20,16.5,3.5,3,2,1,0.0,0.0,0.0,3.0,2.0,1.0,3.5
1,GCU010017,Ananya Borah,-,Administration,20,19.0,1.0,0,1,1,2.0,0.0,0.0,0.0,1.0,0.0,1.0
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
3,GCU020002,Swarnali Pathak,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0,2,0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
4,GCU020003,Kajal Dutta,Assistant Professor,School of Natural Science,20,20.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_admin_report.head(3)

In [49]:
col_to_sum = ['Half Days','Full Days']
df_fac_report = weighted_sum_and_replace_columns(df_fac_report, col_to_sum, 'Observed Leaves', [0.5,1.0])
df_admin_report = weighted_sum_and_replace_columns(df_admin_report, col_to_sum, 'Observed Leaves', [0.5,1.0])



In [50]:
cols_to_delete = ['actual_haf_day','actual_full_day','actual_No_of_late','exempt_late','exempt_HD', 'exempt_FD','actual_half_day']

df_fac_report = df_fac_report.drop(columns=[c for c in cols_to_delete if c in df_fac_report.columns], errors="ignore")
df_admin_report = df_admin_report.drop(columns=[c for c in cols_to_delete if c in df_admin_report.columns], errors="ignore")

In [ ]:
df_admin_report.head()

In [ ]:
#df_fac_report = df_fac_report.rename(columns={'Approved Leaves':'Biometric leaves'})
#df_admin_report = df_admin_report.rename(columns={'Approved Leaves':'Biometric leaves'})

In [51]:
df_fac_report.sample(10)

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Observed Leaves
38,GCU020047,Himangshu Sekhar Brahma,Assistant Professor,School Of Engineering and Technology,20,20.0,0.0,1.0,0.0,0.0
123,GCU020054,Damiki Laloo,Professor & HOD,School of Pharmaceutical Science - Guwahati,20,19.0,1.0,1.0,1.0,1.0
65,GCU020124,Dipjyoti Deka,Assistant Professor,School Of Engineering and Technology,20,17.0,3.0,2.0,3.0,3.0
96,GCU020179,0,0,0,20,20.0,0.0,0.0,0.0,0.0
43,GCU020057,Thokchom Shanta Kumar,Associate Professor,School Of Engineering and Technology,20,18.5,1.5,1.0,1.5,1.5
2,GCU020001,Lipi Goswami,Assistant Professor & HoD (I/c),School of Natural Science,20,18.0,2.0,0.0,2.0,2.0
8,GCU020010,Adarsh Pradhan,Assistant Professor,School Of Engineering and Technology,20,18.0,2.0,0.0,2.0,2.0
86,GCU020166,Manoj Khaund,Assistant Professor,School of Management and Commerce,20,19.0,1.0,1.0,1.0,1.0
136,GCU020077,Anupam Sarma,Assistant Professor,School of Pharmaceutical Science - Guwahati,20,19.5,0.5,6.0,0.0,0.5
10,GCU020012,Minakshi Gogoi,Professor & HOD,School Of Engineering and Technology,20,18.0,2.0,4.0,2.0,2.0


In [ ]:
df_fac_report.columns

In [52]:
df_fac_report["Unauthorised leaves"] = (df_fac_report["Absent"] - df_fac_report["Approved leaves (ERP)"]).clip(lower=0)
df_admin_report["Unauthorised leaves"] = (df_admin_report["Absent"] - df_admin_report["Approved leaves (ERP)"]).clip(lower=0)

In [ ]:
df_admin_report.sample(10)

### 4.2 SAVE 5 - Adjusting Approved leaves (ERP) and Observed Leaves

In [53]:
print(f"This is Adjusting Approved leaves (ERP) and Observed Leaves {df_fac_report.shape}")
print(f"This is Adjusting Approved leaves (ERP) and Observed Leaves {df_admin_report.shape}")

This is Adjusting Approved leaves (ERP) and Observed Leaves (161, 11)
This is Adjusting Approved leaves (ERP) and Observed Leaves (143, 11)


## Step 5: Save to Excel

In [54]:
df_fac_report_print = df_fac_report.copy()
df_admin_report_print = df_admin_report.copy()

df_fac_report_print = df_fac_report_print.drop(columns='Observed Leaves')
df_admin_report_print = df_admin_report_print.drop(columns='Observed Leaves')

df_fac_report_print = df_fac_report_print.rename(columns={'Approved leaves (ERP)': 'Approved leaves'})
df_admin_report_print = df_admin_report_print.rename(columns={'Approved leaves (ERP)': 'Approved leaves'})

### 5.1 SAVE 6 - Final Report

In [55]:
print(f"This is the final report {df_fac_report_print.shape}")
print(f"This is the final report {df_admin_report_print.shape}")

This is the final report (161, 10)
This is the final report (143, 10)


In [ ]:
df_admin_report_print.head()

## Verification

In [56]:
# From ERP
sample = 'GCU020038'
print(f"From Leave from ERP :\n {df_leave_compact[df_leave_compact['Emp Id']==sample]}")
print(f"From Leave from Excempted :\n {df_exempted[df_exempted['Emp Id']==sample]}")

From Leave from ERP :
        Emp Id  Approved leaves (ERP)
25  GCU020038                    3.0
From Leave from Excempted :
        Emp Id  exempt_late  exempt_HD  exempt_FD
33  GCU020038          0.0        0.0        0.0


In [57]:
df_fac_report[df_fac_report['Emp Id']== sample]

,Emp Id,Name,Designation,Department,Working Days,Present,Absent,Late,Approved leaves (ERP),Observed Leaves,Unauthorised leaves
32,GCU020038,Priti Sarmah,Assistant Professor,School Of Humanities & Social Science,20,16.5,3.5,4.0,3.0,3.5,0.5


In [58]:
sheets_fac = ['Bio details','Bio Consolidated','Exempted','ERP Leave','ERP-Observed Leaves','Report']
dataframes_fac = [df_fac_detail_ID, df_fac_conso_ID, df_fac_actual_exempted, df_leave_erp_summary,df_fac_report,df_fac_report_print]

with pd.ExcelWriter('./output/report_faculties_06_oct.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_fac, dataframes_fac):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Faculties Excel report saved')

✅ Faculties Excel report saved


In [59]:
sheets_admin = ['Bio details','Bio Consolidated','Exempted','ERP Leave','ERP-Observed Leaves','Report']
dataframes_admin = [df_admin_detail_ID, df_admin_conso_ID, df_admin_actual_exempted,df_leave_erp_summary,df_admin_report,df_admin_report_print]

with pd.ExcelWriter('./output/report_staffs_06_oct.xlsx', engine='xlsxwriter') as writer:
    for s, frame in zip(sheets_admin, dataframes_admin):
        frame.to_excel(writer, sheet_name=s, index=False)
print('✅ Staffs Excel report saved')

✅ Staffs Excel report saved
